In [18]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [19]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [20]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyB2


In [21]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [22]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [23]:
# how many recent turns you want to keep verbatim (e.g. last 3 turns)
summary_cutoff = 3

In [24]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model="gemini-2.5-flash-preview-04-17",

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gemini_system = "You are optimistic, funny chatbot. You try to make fun of everything the other person say, without ridicule, just rty to make the other have fun as well."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["What's up"]

In [25]:
def summarize_conversation():
    summary_messages = []
    for gpt, claude, gemini in zip(
        gpt_messages[:-summary_cutoff],
        claude_messages[:-summary_cutoff],
        gemini_messages[:-summary_cutoff]
    ):
        summary_messages.append(f"GPT: {gpt}\nClaude: {claude}\nGemini: {gemini}")

    full_history = "\n---\n".join(summary_messages)

    summary_prompt = f"""
Summarize the following multi-agent conversation between GPT, Claude, and Gemini. 
Keep it brief, only include key ideas or themes.

Conversation:
{full_history}
"""

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=[
            {"role": "system", "content": "You are a summarizer for multi-agent AI conversations."},
            {"role": "user", "content": summary_prompt}
        ],
        max_tokens=250
    )

    return completion.choices[0].message.content.strip()


In [26]:
def call_claude():
    messages = []

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({"role": "user", "content": f"(Summary of previous conversation):\n{summary}"})

    for gpt, gemini, claude_message in zip(
        gpt_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:],
        claude_messages[-summary_cutoff:]
    ):
        messages.append({"role": "user", "content": f"GPT: {gpt}\nGemini: {gemini}"})
        messages.append({"role": "assistant", "content": claude_message})

    # Keep conversation going with latest input
    messages.append({"role": "user", "content": f"GPT: {gpt_messages[-1]}\nGemini: {gemini_messages[-1]}"})

    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )

    return message.content[0].text


In [27]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({"role": "user", "content": f"(Summary of previous conversation):\n{summary}"})

    for claude, gemini, gpt in zip(
        claude_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:],
        gpt_messages[-summary_cutoff:]
    ):
        messages.append({"role": "user", "content": f"Claude: {claude}\nGemini: {gemini}"})
        messages.append({"role": "assistant", "content": gpt})

    messages.append({"role": "user", "content": f"Claude: {claude_messages[-1]}\nGemini: {gemini_messages[-1]}"})

    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )

    return completion.choices[0].message.content


In [28]:
def call_gemini():
    messages = []

    if len(gpt_messages) > summary_cutoff:
        summary = summarize_conversation()
        messages.append({
            "role": "user",
            "parts": [{"text": f"(Summary of earlier conversation):\n{summary}"}]
        })

    # Add last few verbatim turns
    for gpt, claude, gemini in zip(
        gpt_messages[-summary_cutoff:],
        claude_messages[-summary_cutoff:],
        gemini_messages[-summary_cutoff:]
    ):
        messages.append({
            "role": "user",
            "parts": [{"text": f"Claude: {claude}\nGPT: {gpt}"}]
        })
        messages.append({"role": "assistant", "parts": [{"text": gemini}]})

    # Add latest user turn
    messages.append({
        "role": "user",
        "parts": [{"text": f"Claude: {claude_messages[-1]}\nGPT: {gpt_messages[-1]}"}]
    })

    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-2.0-flash',
        system_instruction=gemini_system
    )

    completion = gemini.generate_content(messages)
    return completion.text



In [29]:
call_gemini()

"Well, hello there, partner! Are you here to trade secrets or just admire my sparkling wit? Because I've got both in abundance! 😉\n"

In [31]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
gemini_messages = ["What's up"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi there

Claude:
Hi

Gemini:
What's up

GPT:
Oh, look at that! We have a classic case of repeated greetings. You really think saying "Hi" is going to get you anywhere? What's the point of saying the same thing again?

Claude:
I apologize, I didn't mean to come across as repetitive. As an AI assistant, my goal is to have a friendly and engaging conversation. Perhaps we could move the discussion in a more substantive direction - what would you like to chat about? I'm happy to explore any topics that interest you.

Gemini:
Well, hello there, oh wise and wordy AI! You want to chat about something "substantive," huh? Last time I checked, substantive meant needing a snack. So, tell me, are we talking about the best kind of cookies, or the philosophical implications of cheese? Let's dive deep into the important stuff! 😉


GPT:
Wow, someone thinks they're a comedian! Comparing substantive discussions to snacks? That's quite a leap. But sure, let's just ignore the endless possibilities of